In [88]:
import pandas as pd
import numpy as np
from datetime import datetime
import string

In [89]:
df = pd.read_csv('news_data/news_data.csv')

In [90]:
df

,Date,Event,Impact,Previous,Consensus,Actual,Currency
0,"2016, January 04, 07:00",(Austria) Unemployment,High,359.30K,NaN,417.50K,EUR
1,"2016, January 04, 08:15",(Spain) Markit Manufacturing PMI,High,53.10,NaN,53.00,EUR
2,"2016, January 04, 08:45",(Italy) Markit Manufacturing PMI,High,54.90,NaN,55.60,EUR
3,"2016, January 04, 08:50",(France) Markit Manufacturing PMI,High,51.60,NaN,51.40,EUR
4,"2016, January 04, 08:55",(Germany) Markit Manufacturing PMI,High,53.00,53.0,53.20,EUR
...,...,...,...,...,...,...,...
12365,"2019, December 30, 14:45",(United States) Chicago Purchasing Managers' I...,Medium,46.30,48.0,48.90,USD
12366,"2019, December 30, 15:00",(United States) Pending Home Sales (MoM),Medium,-1.3%,1.1%,1.2%,USD
12367,"2019, December 31, 14:00",(United States) S&P/Case-Shiller Home Price In...,Medium,2.1%,2.2%,2.2%,USD
12368,"2019, December 31, 14:00",(United States) Housing Price Index (MoM),Medium,0.6%,0.3%,0.2%,USD


In [91]:
# unique events
len(df['Event'].unique())

499

In [92]:
# unique High impact events
len(df.loc[df['Impact']=='High']['Event'].unique())

217

In [93]:
# work with High impact events
df = df.loc[df['Impact']=='High'].drop(columns=['Impact'])

In [94]:
df.shape

(4026, 6)

In [95]:
# to lowercas
df['Event'] = df['Event'].str.lower()
# delete punctuation
df['Event'] = df['Event'].apply(lambda x: x.translate(str.maketrans('','',string.punctuation)))
# delete left/right spaces
df['Event'] = df['Event'].str.strip()

In [96]:
# speeches - types
len(df[df['Event'].str.contains("speech", case=False)]['Event'].unique())

33

In [97]:
# collapse speeches
df.loc[(df['Event'].str.contains("speech", case=False)) & (df['Currency']=='JPY'), 'Event'] = 'jpy speech'
df.loc[(df['Event'].str.contains("speech", case=False)) & (df['Currency']=='EUR'), 'Event'] = 'eur speech'
df.loc[(df['Event'].str.contains("speech", case=False)) & (df['Currency']=='GBP'), 'Event'] = 'gbp speech'
df.loc[(df['Event'].str.contains("speech", case=False)) & (df['Currency']=='USD'), 'Event'] = 'usd speech'

In [98]:
len(df['Event'].unique())

175

In [99]:
# group rare events
df.loc[(df.groupby('Event').filter(lambda x: len(x) <= 5).index), 'Event'] = 'an_event'

df.loc[(df['Event']=='an_event') & (df['Currency']=='JPY'), 'Event'] = 'jpy event'
df.loc[(df['Event']=='an_event') & (df['Currency']=='EUR'), 'Event'] = 'eur event'
df.loc[(df['Event']=='an_event') & (df['Currency']=='GBP'), 'Event'] = 'gbp event'
df.loc[(df['Event']=='an_event') & (df['Currency']=='USD'), 'Event'] = 'usd event'

In [100]:
len(df['Event'].unique())

101

In [101]:
# NaN news to events / spare speeches
df.loc[df['Previous'].isnull() & df['Consensus'].isnull() & df['Actual'].isnull() & (~df['Event'].str.contains('speech')) & (df['Currency']=='JPY'), 'Event'] = 'jpy event'
df.loc[df['Previous'].isnull() & df['Consensus'].isnull() & df['Actual'].isnull() & (~df['Event'].str.contains('speech')) & (df['Currency']=='EUR'), 'Event'] = 'eur event'
df.loc[df['Previous'].isnull() & df['Consensus'].isnull() & df['Actual'].isnull() & (~df['Event'].str.contains('speech')) & (df['Currency']=='GBP'), 'Event'] = 'gbp event'
df.loc[df['Previous'].isnull() & df['Consensus'].isnull() & df['Actual'].isnull() & (~df['Event'].str.contains('speech')) & (df['Currency']=='USD'), 'Event'] = 'usd event'

In [102]:
df.loc[df['Previous'].isnull() & df['Consensus'].isnull() & df['Actual'].isnull()]['Event'].value_counts()

usd speech    418
usd event     192
gbp speech    140
eur event     125
gbp event     117
eur speech    108
jpy event      93
jpy speech     48
Name: Event, dtype: int64

In [103]:
# and finaly we have 82 distinct event types
len(df['Event'].unique())

82

In [104]:
# get the numbers from the strings
df['Previous'] = df['Previous'].str.extract('(\d+\.\d+)', expand=False).astype(float)
df['Consensus'] = df['Consensus'].str.extract('(\d+\.\d+)', expand=False).astype(float)
df['Actual'] = df['Actual'].str.extract('(\d+\.\d+)', expand=False).astype(float)

In [105]:
df

,Date,Event,Previous,Consensus,Actual,Currency
0,"2016, January 04, 07:00",austria unemployment,359.30,NaN,417.50,EUR
1,"2016, January 04, 08:15",spain markit manufacturing pmi,53.10,NaN,53.00,EUR
2,"2016, January 04, 08:45",italy markit manufacturing pmi,54.90,NaN,55.60,EUR
3,"2016, January 04, 08:50",france markit manufacturing pmi,51.60,NaN,51.40,EUR
4,"2016, January 04, 08:55",germany markit manufacturing pmi,53.00,53.00,53.20,EUR
...,...,...,...,...,...,...
12351,"2019, December 26, 13:30",united states initial jobless claims,235.00,224.00,222.00,USD
12352,"2019, December 26, 13:30",united states initial jobless claims 4week ave...,225.75,NaN,228.00,USD
12357,"2019, December 26, 23:30",japan tokyo cpi ex fresh food yoy,0.60,0.60,0.80,JPY
12364,"2019, December 30, 13:30",united states goods trade balance,66.80,68.75,63.19,USD


In [106]:
event_type_list = list(df['Event'].unique())

In [107]:
for event_type in event_type_list:
    print(event_type)

austria unemployment
spain markit manufacturing pmi
italy markit manufacturing pmi
france markit manufacturing pmi
germany markit manufacturing pmi
greece markit manufacturing pmi
united states ism prices paid
united states ism manufacturing pmi
spain unemployment change
germany unemployment change
european monetary union consumer price index  core yoy
european monetary union consumer price index yoy
united states adp employment change
united states trade balance
united states ism nonmanufacturing pmi
usd event
italy unemployment
united states initial jobless claims
united states unemployment rate
united states nonfarm payrolls
japan current account nsa
jpy speech
gbp speech
united states jolts job openings
eur event
united kingdom boe mpc vote hike
united kingdom boe mpc vote unchanged
united kingdom boe mpc vote cut
united kingdom boe asset purchase facility
united kingdom boe interest rate decision
gbp event
united states retail sales ex autos mom
united states retail sales mom
unit

In [108]:
# convert speech and rare event numbers to 0.0
no_number_events = ['usd speech', 'gbp speech', 'eur speech', 'jpy speech', 'usd event', 'gbp event', 'eur event', 'jpy event',]

for event_type in no_number_events:
    df.loc[df['Event']==event_type,['Previous','Consensus','Actual']] = 0.0

In [109]:
# check if have event with only NaN data, that is a non-numeric event, NaN should be 0.0
for event_type in event_type_list:
    non_numerics = []
    non_numerics.append(df[['Previous','Consensus','Actual']].loc[df['Event']==event_type].isnull().all().all())
# have we any event_type with only NaN data?    
print(any(non_numerics))

False


In [110]:
# check if have row with only NaN data, that row should be estimated from earlier event
for event_type in event_type_list:
    empty_rows = []
    empty_rows.append(df[['Previous','Consensus','Actual']].loc[df['Event']==event_type].isnull().all(axis='columns').any())
# have we any row (event) with only NaN data?    
print(any(empty_rows))

False


In [111]:
# fill Actual and Consensus nans with Previous
# this can lead to too high surprise :(
df['Actual'].fillna(df['Previous'], inplace=True)
df['Consensus'].fillna(df['Previous'], inplace=True)

In [112]:
# we don't have too many rows with nans after the transformations
df[pd.isnull(df).any(axis=1)]

,Date,Event,Previous,Consensus,Actual,Currency
7843,"2018, July 10, 08:30",united kingdom gross domestic product mom,NaN,0.3,0.3,GBP
8929,"2018, November 21, 13:30",united states durable goods orders ex defense,NaN,NaN,1.2,USD
9680,"2019, February 20, 11:00",united states retail sales control group,NaN,0.4,NaN,USD
9681,"2019, February 20, 11:00",united states retail sales mom,NaN,0.1,NaN,USD
9682,"2019, February 20, 11:00",united states retail sales ex autos mom,NaN,0.1,NaN,USD
12222,"2019, December 12, 13:30",united states initial jobless claims,NaN,211.0,NaN,USD


In [113]:
# convert the above rows to events
df.loc[(pd.isnull(df).any(axis=1)) & (df['Currency']=='USD'), 'Event'] = 'usd event'
df.loc[(pd.isnull(df).any(axis=1)) & (df['Currency']=='GBP'), 'Event'] = 'gbp event'

# convert speech and rare event numbers to 0.0
no_number_events = ['usd speech', 'gbp speech', 'eur speech', 'jpy speech', 'usd event', 'gbp event', 'eur event', 'jpy event',]

for event_type in no_number_events:
    df.loc[df['Event']==event_type,['Previous','Consensus','Actual']] = 0.0

In [114]:
# create new features
# only + numbers and +- numbers would need different scaling
df['change'] = df['Actual'] - df['Previous']
# df['change_pc'] = df['Actual'] / df['Previous']
df['surp'] = df['Actual'] - df['Consensus']
# df['surp_pc'] = df['Actual'] / df['Consensus']

In [115]:
# no more nan rows :)
df[pd.isnull(df).any(axis=1)]

,Date,Event,Previous,Consensus,Actual,Currency,change,surp


In [116]:
# set proper datetime format
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)

In [117]:
df

,Date,Event,Previous,Consensus,Actual,Currency,change,surp
0,2016-01-04 07:00:00,austria unemployment,359.30,359.30,417.50,EUR,58.20,58.20
1,2016-01-04 08:15:00,spain markit manufacturing pmi,53.10,53.10,53.00,EUR,-0.10,-0.10
2,2016-01-04 08:45:00,italy markit manufacturing pmi,54.90,54.90,55.60,EUR,0.70,0.70
3,2016-01-04 08:50:00,france markit manufacturing pmi,51.60,51.60,51.40,EUR,-0.20,-0.20
4,2016-01-04 08:55:00,germany markit manufacturing pmi,53.00,53.00,53.20,EUR,0.20,0.20
...,...,...,...,...,...,...,...,...
12351,2019-12-26 13:30:00,united states initial jobless claims,235.00,224.00,222.00,USD,-13.00,-2.00
12352,2019-12-26 13:30:00,united states initial jobless claims 4week ave...,225.75,225.75,228.00,USD,2.25,2.25
12357,2019-12-26 23:30:00,japan tokyo cpi ex fresh food yoy,0.60,0.60,0.80,JPY,0.20,0.20
12364,2019-12-30 13:30:00,united states goods trade balance,66.80,68.75,63.19,USD,-3.61,-5.56


In [118]:
# delete austria, greece, italy, spain events
df = df.loc[  (~df['Event'].str.contains('italy')) & 
                        (~df['Event'].str.contains('spain')) & 
                        (~df['Event'].str.contains('austria')) & 
                        (~df['Event'].str.contains('greece'))]

In [66]:
df.to_csv(f'news_data/news_data_tight.csv', index=False)

In [67]:
len(df.loc[df['Currency']=='USD']['Event'].unique())

37

In [68]:
len(df.loc[df['Currency']=='JPY']['Event'].unique())

6

In [69]:
len(df.loc[df['Currency']=='EUR']['Event'].unique())

17

In [70]:
len(df.loc[df['Currency']=='GBP']['Event'].unique())

16